In [1]:
from transformers.models.efficientformer.modeling_efficientformer import *
import torch
import re
import pathlib
from transformers import ViTFeatureExtractor
from PIL import Image
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

In [2]:
cp = torch.load('D:/efficientformer_l1_300d.pth', map_location = torch.device('cpu'))['model']

In [3]:
def rename_key(old_name):
    new_name = old_name
    
    if 'patch_embed' in old_name:
        _, layer, param = old_name.split('.')
        
        if layer == '0':
            new_name = old_name.replace('0', 'conv1')
        elif layer == '1':
            new_name = old_name.replace('1', 'norm1')
        elif layer == '3':
            new_name = old_name.replace('3', 'conv2')
        else:
            new_name = old_name.replace('4', 'norm2')
            
    if 'network' in old_name and re.search('\d\.\d', old_name):
        match = re.search('\d\.\d', old_name).group()
        new_name = old_name.replace(match, match[:2] + 'blocks.' + match[2:]) 
    
    if 'proj' in old_name:
        new_name = new_name.replace('proj', 'projection')
        
    if 'dist_head' in new_name:
        pass
    elif 'head' in new_name:
        new_name = new_name.replace('head', 'classifier')
    elif 'patch_embed' in new_name or new_name == 'norm.weight' or new_name == 'norm.bias':
        new_name = 'efficientformer.' + new_name
    else:
        new_name = 'efficientformer.encoder.' + new_name
        
    return new_name

In [4]:
def convert_checkpoint(checkpoint):
    for key in checkpoint.copy().keys():
        val = checkpoint.pop(key)
        checkpoint[rename_key(key)] = val
    
    return checkpoint

In [5]:
cp = convert_checkpoint(cp)

In [6]:
config = EfficientformerConfig(num_labels = 1000)
model = EfficientformerForImageClassification(config)

In [7]:
model.load_state_dict(cp)

<All keys matched successfully>

In [8]:
image = torch.zeros([1, 3, 224, 224])

In [10]:
def prepare_img():
    image = Image.open("C:\\Users\\prana\\transformers\\tests\\fixtures\\tests_samples\\COCO\\000000039769.png")
    return image

feature_extractor = ViTFeatureExtractor()
image = prepare_img()
inputs = feature_extractor(images=image, return_tensors="pt").to(torch.device('cpu'))

In [11]:
with torch.no_grad():
    out = model(**inputs).logits

In [14]:
out[1].shape

torch.Size([1000])

In [17]:
out[0][:3], out[1][:3]

(tensor([-0.8568,  0.3356, -0.1594]), tensor([-1.4141,  1.7621,  0.5935]))